In [2]:
from datasets import  concatenate_datasets, load_from_disk
org_dataset = load_from_disk('./data/train_dataset')
print(org_dataset)
full_ds = concatenate_datasets(
        [
            org_dataset["train"].flatten_indices(),
            org_dataset["validation"].flatten_indices(),
        ]
    )  # train dev 를 합친 4192 개 질문에 대해 모두 테스트
print("*" * 40, "query dataset", "*" * 40)
print(full_ds)

c:\Users\승범 pc\Desktop\study\deeplearning\naver_boot\level2-mrc-nlp-07\level2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})
**************************************** query dataset ****************************************
Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 4192
})


In [3]:
full_ds['answers'][0], full_ds['context'][0][235:]

({'answer_start': [235], 'text': ['하원']},
 '하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05')

In [5]:
correct_count = 0
incorrect_count = 0

for i in range(len(full_ds)):
    sample = full_ds[i]
    context = sample['context']
    answers = sample['answers']
    
    for answer in answers['text']:
        start = answers['answer_start'][0]
        extracted_answer = context[start:start + len(answer)]
        
        if extracted_answer == answer:
            correct_count += 1
        else:
            incorrect_count += 1

total = correct_count + incorrect_count
accuracy = (correct_count / total) * 100 if total > 0 else 0

print(f"총 샘플 수: {total}")
print(f"정확한 위치에 있는 답변 수: {correct_count}")
print(f"부정확한 위치에 있는 답변 수: {incorrect_count}")
print(f"정확도: {accuracy:.2f}%")

총 샘플 수: 4192
정확한 위치에 있는 답변 수: 4192
부정확한 위치에 있는 답변 수: 0
정확도: 100.00%


In [12]:
import textwrap
from IPython.display import clear_output

def review_dataset(dataset, batch_size=2):
    total_samples = len(dataset)
    current_index = 7

    while current_index < total_samples:
        clear_output(wait=True)
        for i in range(current_index, min(current_index + batch_size, total_samples)):
            sample = dataset[i]
            start = sample['answers']['answer_start'][0]
            print(f"\n샘플 {i+1}/{total_samples}")
            print(f"제목: {sample['title']}")
            print(f"질문: {sample['question']}")
            print("내용:")
            print(textwrap.fill(sample['context'], width=60))
            print(f"답변: {sample['answers']['text'][0]}")

            answer_context = sample['context'][start:start+20]
            print(f"답변 위치: {answer_context}")
            
            print("-" * 100)

        user_input = input("계속하려면 'g'를, 중지하려면 's'를 입력하세요: ").lower()
        if user_input == 's':
            print("검토를 중지합니다.")
            break
        elif user_input != 'g':
            print("잘못된 입력입니다")
            continue

        current_index += batch_size

    print("Fin")

# 데이터셋을 불러온 후 함수 호출
review_dataset(full_ds)


샘플 8/4192
제목: 제50노섬브리안 보병사단
질문: 참호 속에 무기와 장비를 버리고 도주한 집단은?
내용:
이집트로 이어지는 해안 도로는 1개의 사단만이 방어할 수 있었으며 엘 아뎀 요새와 남아있는 영국 기갑군
잔병들에 의해서만 출입이 가능했다. 이러한 임무는 남아프리카 연방군에게도 할당되었다. 제50사단은 동쪽에서
싸우는 대안으로 좌측을 선택하여 독일 기갑 사단의 편제를 통과하거나 이탈리아 기갑사단 주위로 긴 방어선을
형성하는 것을 목표로 했다. 그들 모두를 파괴하는 것은 거의 불가능했는데, 그 이유는 사단은 혼성 부대로
이루어져 있었기 때문이다. 제5이스트요크셔와 제9더럼경보병부대가 형성한 교두보를 담당하여 이탈리아 전선으로
진입하는 것이 그들의 임무였다. 혼란과 혼돈 속에서 혼성 부대는 독일군이 장악한 동쪽보다 남쪽으로 이동하여
이집트 국경인 마델레나 요새로 이동했다. text=교두보에 있었던 적은 독일군 포병이 지원하는 이탈리아
군이었다. 그들은 기습으로 매우 놀란 것 같았다. 그들이 전 사단이 그들의 전선을 곧바로 통과하는 것을
깨달았는데 밤이 되기 전이었다. 몇몇 차량은 지뢰로 파괴되고, 다른 차량들은 불에 탔지만, 우리의 모든
사단은 적은 사상자를 냈고, 공격대대도 그들의 임무를 성공적으로 마쳤다. 보병은 마차에 탔고, 이탈리아군은
참호에 그들의 무기와 장비를 남겨둔 채 도망갔다. 더럼경보병여단의 제9대대와 제6대대의 소규모 부대는
제69여단 요새 뒤에서 주둔한 이후 해안도로를 따라 후퇴하면서 이탈리아 군인들이 돌파를 알리는 것을 보았다.
독일군 포병 및 보병과 남아프리카 공화국 보호부대의 실수로 공격을 받기는 했으나 혼성 부대는 독일군을
돌파하며 포로까지 잡았다. 6월 17일에서 18일까지 이들은 비르 엘 타라타에서 재집결했다.
답변: 이탈리아군
답변 위치: 이탈리아군은 참호에 그들의 무기와 장
---------------------------------------------------------------------------------